# 판다스 기본
* [해당 데이데 출처- 공공데이터 포탈](https://www.data.go.kr/tcs/opd/ndm/view.do)

In [1]:
import warnings
warnings.simplefilter(action = 'ignore', category=FutureWarning)

from matplotlib import font_manager, rc
font_name  = font_manager.FontProperties(fname = "c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family = font_name)
# 마이너스 부호 표시 
plt.rcParams['axes.unicode_minus'] = False

### pandas 모듈 임포트 

In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
pd.__version__

'1.1.5'

### csv파일 불러오기

In [40]:
# 파이썬의 기본 인코딩 : utf-8이므로 에러가 남 UnicodeDecodeError 
# 이런 경우엔 cpc949
df = pd.read_csv("./data/민간 아파트 분양가격 동향_20200331.csv", encoding = 'CP949')

#### 살펴보기 

In [5]:
df.head(n = 2)

,지역명,규모구분,연도,월,분양가격(㎡)
0,서울,전체,2015,10,5841
1,서울,전용면적 60㎡이하,2015,10,5652


In [6]:
df.tail(n = 2)

,지역명,규모구분,연도,월,분양가격(㎡)
4588,제주,전용면적 85㎡초과 102㎡이하,2020,3,NaN
4589,제주,전용면적 102㎡초과,2020,3,3601


### 열의 이름 바꾸기

In [7]:
df.rename(columns = {'분양가격(㎡)': '분양가'},inplace = True, errors = 'ignore')

In [8]:
df.columns = ['지역', '규모', '연도', '월', '분양가']

In [9]:
df.head()

,지역,규모,연도,월,분양가
0,서울,전체,2015,10,5841
1,서울,전용면적 60㎡이하,2015,10,5652
2,서울,전용면적 60㎡초과 85㎡이하,2015,10,5882
3,서울,전용면적 85㎡초과 102㎡이하,2015,10,5721
4,서울,전용면적 102㎡초과,2015,10,5879


### data type보기

In [10]:
df.dtypes

지역     object
규모     object
연도      int64
월       int64
분양가    object
dtype: object

#### 아래 데이터 변경 메소드는 `1.0.3`에서는 지원하지 않음.
* [스택오버플로우](https://stackoverflow.com/questions/33126477/pandas-convert-objectsconvert-numeric-true-deprecated)

In [11]:
# 해당 메소드는 1.0.3에서는 지원하지 않음.
df['분양가'] = df['분양가'].conver_objects(convert_numeric=True)

AttributeError: 'Series' object has no attribute 'conver_objects'

In [37]:
# 그래서 이렇게 했더니 다음과 같은 에러가 나옴
df['분양가'] = pd.to_numeric(df['분양가'], errors = 'raise')

스택오버플로우에서 검색해보면, 이런 경우는 바꿀려고 하는 데이터타입에 변환할수 없는 널값이나 빈 문자열이 들어가있어서 그렇다. 
이런 경우엔 이렇게 변환해보자.
[관련 스택오버플로우](
https://stackoverflow.com/questions/40790031/pandas-to-numeric-find-out-which-string-it-was-unable-to-parse)

* `errors = 'coerce'` 옵션은 변환할수 없는 데이터타입은 NaN으로 처리.

In [38]:
df['분양가'] = pd.to_numeric(df['분양가'], errors = 'ignore')

* 에러가 나더라도 무시한다.

In [39]:
df['분양가'] = pd.to_numeric(df['분양가'],errors = 'coerce')

In [15]:
df.dtypes

지역      object
규모      object
연도       int64
월        int64
분양가    float64
dtype: object

## numpy array로 변환하기

In [16]:
arr = df.to_numpy()

In [17]:
arr.shape, arr.shape[0], len(arr)

((4590, 5), 4590, 4590)

In [18]:
len(arr), arr.shape

(4590, (4590, 5))

In [19]:
arr[0], arr[1]  # 첫번째 로우,  두번째 로우 

(array(['서울', '전체', 2015, 10, 5841.0], dtype=object),
 array(['서울', '전용면적 60㎡이하', 2015, 10, 5652.0], dtype=object))

### 간단한 통계보기

In [20]:
df.describe()

,연도,월,분양가
count,4590.000000,4590.000000,4260.000000
mean,2017.500000,6.500000,3270.160798
std,1.343856,3.594368,1300.362742
min,2015.000000,1.000000,1868.000000
25%,2016.000000,3.000000,2454.750000
50%,2017.500000,6.500000,2890.000000
75%,2019.000000,10.000000,3601.000000
max,2020.000000,12.000000,13835.000000


### Transposing하기(축변환)

In [21]:
df.head(n = 3)

,지역,규모,연도,월,분양가
0,서울,전체,2015,10,5841.0
1,서울,전용면적 60㎡이하,2015,10,5652.0
2,서울,전용면적 60㎡초과 85㎡이하,2015,10,5882.0


In [22]:
df.transpose().head(2)  # `지역`컬럼의 값만큼 컬럼이 옆으로 늘여져 생성

,0,1,2,3,4,5,6,7,8,9,...,4580,4581,4582,4583,4584,4585,4586,4587,4588,4589
지역,서울,서울,서울,서울,서울,인천,인천,인천,인천,인천,...,경남,경남,경남,경남,경남,제주,제주,제주,제주,제주
규모,전체,전용면적 60㎡이하,전용면적 60㎡초과 85㎡이하,전용면적 85㎡초과 102㎡이하,전용면적 102㎡초과,전체,전용면적 60㎡이하,전용면적 60㎡초과 85㎡이하,전용면적 85㎡초과 102㎡이하,전용면적 102㎡초과,...,전체,전용면적 60㎡이하,전용면적 60㎡초과 85㎡이하,전용면적 85㎡초과 102㎡이하,전용면적 102㎡초과,전체,전용면적 60㎡이하,전용면적 60㎡초과 85㎡이하,전용면적 85㎡초과 102㎡이하,전용면적 102㎡초과


In [23]:
df.T.head(2)

,0,1,2,3,4,5,6,7,8,9,...,4580,4581,4582,4583,4584,4585,4586,4587,4588,4589
지역,서울,서울,서울,서울,서울,인천,인천,인천,인천,인천,...,경남,경남,경남,경남,경남,제주,제주,제주,제주,제주
규모,전체,전용면적 60㎡이하,전용면적 60㎡초과 85㎡이하,전용면적 85㎡초과 102㎡이하,전용면적 102㎡초과,전체,전용면적 60㎡이하,전용면적 60㎡초과 85㎡이하,전용면적 85㎡초과 102㎡이하,전용면적 102㎡초과,...,전체,전용면적 60㎡이하,전용면적 60㎡초과 85㎡이하,전용면적 85㎡초과 102㎡이하,전용면적 102㎡초과,전체,전용면적 60㎡이하,전용면적 60㎡초과 85㎡이하,전용면적 85㎡초과 102㎡이하,전용면적 102㎡초과


### 정렬

In [24]:
df.head()

,지역,규모,연도,월,분양가
0,서울,전체,2015,10,5841.0
1,서울,전용면적 60㎡이하,2015,10,5652.0
2,서울,전용면적 60㎡초과 85㎡이하,2015,10,5882.0
3,서울,전용면적 85㎡초과 102㎡이하,2015,10,5721.0
4,서울,전용면적 102㎡초과,2015,10,5879.0


#### 1.index정렬

In [25]:
# 내림차순 정렬, 상위 5개 
df.sort_index(axis = 0, ascending = False) [:5]  

,지역,규모,연도,월,분양가
4589,제주,전용면적 102㎡초과,2020,3,3601.0
4588,제주,전용면적 85㎡초과 102㎡이하,2020,3,NaN
4587,제주,전용면적 60㎡초과 85㎡이하,2020,3,3962.0
4586,제주,전용면적 60㎡이하,2020,3,4039.0
4585,제주,전체,2020,3,3955.0


In [26]:
#올림차순 정렬
df.sort_index(axis = 0, ascending = True)[:5]

,지역,규모,연도,월,분양가
0,서울,전체,2015,10,5841.0
1,서울,전용면적 60㎡이하,2015,10,5652.0
2,서울,전용면적 60㎡초과 85㎡이하,2015,10,5882.0
3,서울,전용면적 85㎡초과 102㎡이하,2015,10,5721.0
4,서울,전용면적 102㎡초과,2015,10,5879.0


#### 2. Value정렬

In [27]:
# 한 개이상의 변수로 정렬
df.sort_values(by = ['지역', '연도'])[:5]

,지역,규모,연도,월,분양가
45,강원,전체,2015,10,2167.0
46,강원,전용면적 60㎡이하,2015,10,2286.0
47,강원,전용면적 60㎡초과 85㎡이하,2015,10,2212.0
48,강원,전용면적 85㎡초과 102㎡이하,2015,10,2061.0
49,강원,전용면적 102㎡초과,2015,10,2171.0


In [28]:
# `연도`로 정렬
df.sort_values(by = '연도')[:5]

,지역,규모,연도,월,분양가
0,서울,전체,2015,10,5841.0
162,경남,전용면적 60㎡초과 85㎡이하,2015,11,2322.0
163,경남,전용면적 85㎡초과 102㎡이하,2015,11,2980.0
164,경남,전용면적 102㎡초과,2015,11,3043.0
165,제주,전체,2015,11,2232.0


### Selection(선택)

#### 1. Column 이름으로 선택¶

In [29]:
df['지역'][:5], type(df['지역'][:5])

(0    서울
 1    서울
 2    서울
 3    서울
 4    서울
 Name: 지역, dtype: object,
 pandas.core.series.Series)

#### 2.Index선택

In [30]:
df[0:5] # 인덱스 0부터 인덱스 4까지 

,지역,규모,연도,월,분양가
0,서울,전체,2015,10,5841.0
1,서울,전용면적 60㎡이하,2015,10,5652.0
2,서울,전용면적 60㎡초과 85㎡이하,2015,10,5882.0
3,서울,전용면적 85㎡초과 102㎡이하,2015,10,5721.0
4,서울,전용면적 102㎡초과,2015,10,5879.0


In [31]:
df[-5:]  # 맨뒤에서 5번째부터 끝까지

,지역,규모,연도,월,분양가
4585,제주,전체,2020,3,3955.0
4586,제주,전용면적 60㎡이하,2020,3,4039.0
4587,제주,전용면적 60㎡초과 85㎡이하,2020,3,3962.0
4588,제주,전용면적 85㎡초과 102㎡이하,2020,3,NaN
4589,제주,전용면적 102㎡초과,2020,3,3601.0


In [32]:
# 맨 뒤의 것 하나만 출력 
df[-1:]

,지역,규모,연도,월,분양가
4589,제주,전용면적 102㎡초과,2020,3,3601.0


#### 3. Label로 선택
* loc - 명칭기반 인덱싱

In [41]:
df.loc[:, ['지역명', '연도']][:5]  # 5개만 출력

,지역명,연도
0,서울,2015
1,서울,2015
2,서울,2015
3,서울,2015
4,서울,2015


In [42]:
df.loc[:6, ['지역명', '연도']] # 앞에 6번째 인덱스가 포함됨에 유의할것.

,지역명,연도
0,서울,2015
1,서울,2015
2,서울,2015
3,서울,2015
4,서울,2015
5,인천,2015
6,인천,2015


In [43]:
# 인덱스가 5보다 큰것을 가져오는데 10개만 출력
cond1 = df.index > 5
df.loc[cond1, ['지역명', '연도']][:10]

,지역명,연도
6,인천,2015
7,인천,2015
8,인천,2015
9,인천,2015
10,경기,2015
11,경기,2015
12,경기,2015
13,경기,2015
14,경기,2015
15,부산,2015


In [44]:
df.loc[df['지역명'] == '인천', ['지역명', '연도']][:5] #지역명 column의 값이 '인천'인 행의 '지역명', '연도' 출력

,지역명,연도
5,인천,2015
6,인천,2015
7,인천,2015
8,인천,2015
9,인천,2015


#### 4. iloc을 활용한 인덱스 지정 선택
* 위치기반 인덱싱(행을 slicing시에 `:`기준으로 뒤의 인덱스가 결과에 포함되지 않음에 유의)
* `,`기준으로 앞은 행, 뒤는 열을 의미
* 인덱스를 명시적으로 지정하면 해당 인덱스를 의미함.(인덱스는 0부터 시작함을 유의)

In [45]:
df.head()

,지역명,규모구분,연도,월,분양가격(㎡)
0,서울,전체,2015,10,5841
1,서울,전용면적 60㎡이하,2015,10,5652
2,서울,전용면적 60㎡초과 85㎡이하,2015,10,5882
3,서울,전용면적 85㎡초과 102㎡이하,2015,10,5721
4,서울,전용면적 102㎡초과,2015,10,5879


In [46]:
df.iloc[2, 1]  # 행 인덱스가 2이고 , 열인덱스가 1(규모구분)인 것을 가져오니까..

'전용면적 60㎡초과 85㎡이하'

In [47]:
# df.iloc과 동작방식은 동일하지만, 범위 지정은 불가
df.iat[2, 1]

'전용면적 60㎡초과 85㎡이하'

In [48]:
df.iat[:2, 1]  #  범위 지정은 안됨. ValueError: iAt based indexing can only have integer indexers

ValueError: iAt based indexing can only have integer indexers

In [49]:
# 3번째 인덱스를 가져오되, 열정보는 모두 다 가져옴.
df.iloc[3, :], type(df.iloc[3, :])

(지역명                       서울
 규모구분       전용면적 85㎡초과 102㎡이하
 연도                      2015
 월                         10
 분양가격(㎡)                 5721
 Name: 3, dtype: object,
 pandas.core.series.Series)

In [50]:
# 위치기반 인덱스인 iloc는 마지막인덱스는 포함되지 않음에 유의.
# iloc으로 인덱스 지정시 : 기준으로 왼쪽은 포함, : 기준으로 오른쪽은 미만
df.iloc[:4,1:3]

,규모구분,연도
0,전체,2015
1,전용면적 60㎡이하,2015
2,전용면적 60㎡초과 85㎡이하,2015
3,전용면적 85㎡초과 102㎡이하,2015


In [51]:
# IndexError: .iloc requires numeric indexers, got ['규모구분' '연도']
# 위의 사례처럼 해야함.
df.iloc[:4, ['규모구분','연도']]  

IndexError: .iloc requires numeric indexers, got ['규모구분' '연도']

In [52]:
df.iloc[ : 5, 1: 3]  # 4번째 행 인덱스까지 가져오고, 열은 1번 인덱스부터 2번 인덱스까지.

,규모구분,연도
0,전체,2015
1,전용면적 60㎡이하,2015
2,전용면적 60㎡초과 85㎡이하,2015
3,전용면적 85㎡초과 102㎡이하,2015
4,전용면적 102㎡초과,2015


#### 5. 범위 조건 지정 선택

In [53]:
df[df.index > 3565]

,지역명,규모구분,연도,월,분양가격(㎡)
3566,제주,전용면적 60㎡이하,2019,3,3804
3567,제주,전용면적 60㎡초과 85㎡이하,2019,3,3504
3568,제주,전용면적 85㎡초과 102㎡이하,2019,3,3226
3569,제주,전용면적 102㎡초과,2019,3,2952
3570,서울,전체,2019,4,7784
...,...,...,...,...,...
4585,제주,전체,2020,3,3955
4586,제주,전용면적 60㎡이하,2020,3,4039
4587,제주,전용면적 60㎡초과 85㎡이하,2020,3,3962
4588,제주,전용면적 85㎡초과 102㎡이하,2020,3,NaN


#### 5-a. df.연도  와  df['연도'] 의 column 지정방식은 동일하다

In [54]:
df[df.연도 == 2019][:5]

,지역명,규모구분,연도,월,분양가격(㎡)
3315,서울,전체,2019,1,7600
3316,서울,전용면적 60㎡이하,2019,1,7400
3317,서울,전용면적 60㎡초과 85㎡이하,2019,1,8105
3318,서울,전용면적 85㎡초과 102㎡이하,2019,1,6842
3319,서울,전용면적 102㎡초과,2019,1,7787


In [55]:
df[df['연도'] == 2019][:5]

,지역명,규모구분,연도,월,분양가격(㎡)
3315,서울,전체,2019,1,7600
3316,서울,전용면적 60㎡이하,2019,1,7400
3317,서울,전용면적 60㎡초과 85㎡이하,2019,1,8105
3318,서울,전용면적 85㎡초과 102㎡이하,2019,1,6842
3319,서울,전용면적 102㎡초과,2019,1,7787


### Copy로 복사
* 데이터프레임을 할당연산자로 복사하게되면, 원치않는 결과가 만들어지니 ***반드시 `.copy()`명령어를 사용***해라
* 할당 연산자로 복사하게 되면, 참조하는 주소를 공유하게 되므로 위험한 결과를 초래함.

In [56]:
df.head(2)

,지역명,규모구분,연도,월,분양가격(㎡)
0,서울,전체,2015,10,5841
1,서울,전용면적 60㎡이하,2015,10,5652


In [57]:
copy_df = df.copy()

In [ ]:
copy_df['중산층'] = np.nan

In [58]:
display(copy_df.head(2), df.head(2))

,지역명,규모구분,연도,월,분양가격(㎡)
0,서울,전체,2015,10,5841
1,서울,전용면적 60㎡이하,2015,10,5652


,지역명,규모구분,연도,월,분양가격(㎡)
0,서울,전체,2015,10,5841
1,서울,전용면적 60㎡이하,2015,10,5652


In [59]:
oops_df = df

In [60]:
oops_df['중산층'] = np.nan

In [61]:
# oops_df에만 추가하려고 했는데, 두 데이터프레임 모두 변경됨.
display(oops_df.head(2), df.head(2))

,지역명,규모구분,연도,월,분양가격(㎡),중산층
0,서울,전체,2015,10,5841,NaN
1,서울,전용면적 60㎡이하,2015,10,5652,NaN


,지역명,규모구분,연도,월,분양가격(㎡),중산층
0,서울,전체,2015,10,5841,NaN
1,서울,전용면적 60㎡이하,2015,10,5652,NaN


### 값 지정
* label명으로 가져오는 것은 loc만 가능함.

In [62]:
df.loc[df.지역명 == '서울', '지역명'][:5]

0    서울
1    서울
2    서울
3    서울
4    서울
Name: 지역명, dtype: object

In [63]:
# 지역명이 `서울` 인 지역을 `Seoul`로 변경
df.loc[df.지역명 == '서울', '지역명'] = 'seoul'

In [64]:
df.loc[df.지역명 == 'seoul', '지역명'][:5]

0    seoul
1    seoul
2    seoul
3    seoul
4    seoul
Name: 지역명, dtype: object

### reindex를 통한 지정 행과 새로운 열을 추가하여 새로운 dataframe으로 생성

In [65]:
df.index

RangeIndex(start=0, stop=4590, step=1)

In [66]:
type(df.columns)

pandas.core.indexes.base.Index

In [67]:
df1  = df.reindex(index = df.index[:7], columns = list(df.columns) + ['extra'])

In [68]:
df1

,지역명,규모구분,연도,월,분양가격(㎡),중산층,extra
0,seoul,전체,2015,10,5841,NaN,NaN
1,seoul,전용면적 60㎡이하,2015,10,5652,NaN,NaN
2,seoul,전용면적 60㎡초과 85㎡이하,2015,10,5882,NaN,NaN
3,seoul,전용면적 85㎡초과 102㎡이하,2015,10,5721,NaN,NaN
4,seoul,전용면적 102㎡초과,2015,10,5879,NaN,NaN
5,인천,전체,2015,10,3163,NaN,NaN
6,인천,전용면적 60㎡이하,2015,10,3488,NaN,NaN


In [69]:
df1.loc[:4, 'extra'] = False

In [70]:
df1[:5]

,지역명,규모구분,연도,월,분양가격(㎡),중산층,extra
0,seoul,전체,2015,10,5841,NaN,False
1,seoul,전용면적 60㎡이하,2015,10,5652,NaN,False
2,seoul,전용면적 60㎡초과 85㎡이하,2015,10,5882,NaN,False
3,seoul,전용면적 85㎡초과 102㎡이하,2015,10,5721,NaN,False
4,seoul,전용면적 102㎡초과,2015,10,5879,NaN,False


### 빈 데이터 처리

In [71]:
df2 = df1.copy()

In [72]:
df2

,지역명,규모구분,연도,월,분양가격(㎡),중산층,extra
0,seoul,전체,2015,10,5841,NaN,False
1,seoul,전용면적 60㎡이하,2015,10,5652,NaN,False
2,seoul,전용면적 60㎡초과 85㎡이하,2015,10,5882,NaN,False
3,seoul,전용면적 85㎡초과 102㎡이하,2015,10,5721,NaN,False
4,seoul,전용면적 102㎡초과,2015,10,5879,NaN,False
5,인천,전체,2015,10,3163,NaN,NaN
6,인천,전용면적 60㎡이하,2015,10,3488,NaN,NaN


#### 1.NaN이 있는 값 처리

In [74]:
df2.isna()

,지역명,규모구분,연도,월,분양가격(㎡),중산층,extra
0,False,False,False,False,False,True,False
1,False,False,False,False,False,True,False
2,False,False,False,False,False,True,False
3,False,False,False,False,False,True,False
4,False,False,False,False,False,True,False
5,False,False,False,False,False,True,True
6,False,False,False,False,False,True,True


In [76]:
df2[df2.isna()]

,지역명,규모구분,연도,월,분양가격(㎡),중산층,extra
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN


* `how='any'` 옵션은 NaN값이 하나라도 있으면 해당 컬럼혹은 로우를 삭제함.
* 로우 혹은 컬럼의 삭제는 `axis= 0 | 1` 값이 무엇이냐에 따라 다르다.
* dropna의 `axis`의 기본값은 0.
* `how`의 옵션 정보

how : {'any', 'all'}, default 'any'

    Determine if row or column is removed from DataFrame, when we have
    
    at least one NA or all NA.
    
    * 'any' : If any NA values are present, drop that row or column.
    
    * 'all' : If all values are NA, drop that row or column.

In [81]:
df2.dropna(how='any', axis = 0, inplace = False)  # inplace옵션은 기본값이 False,원본에 반영 안함.

,지역명,규모구분,연도,월,분양가격(㎡),중산층,extra


#### 1.a 하지만 원본 데이터에는 반영이 되어있지 않음.

In [82]:
df2

,지역명,규모구분,연도,월,분양가격(㎡),중산층,extra
0,seoul,전체,2015,10,5841,NaN,False
1,seoul,전용면적 60㎡이하,2015,10,5652,NaN,False
2,seoul,전용면적 60㎡초과 85㎡이하,2015,10,5882,NaN,False
3,seoul,전용면적 85㎡초과 102㎡이하,2015,10,5721,NaN,False
4,seoul,전용면적 102㎡초과,2015,10,5879,NaN,False
5,인천,전체,2015,10,3163,NaN,NaN
6,인천,전용면적 60㎡이하,2015,10,3488,NaN,NaN


In [83]:
df2.dropna(how = 'any', inplace = True) # 원본에 반영함. 리턴값은 None

In [84]:
# axis옵션은 기본값이 0이므로 dropna하면 NaN값이 있는 해당 로우가 삭제됨.
df2

,지역명,규모구분,연도,월,분양가격(㎡),중산층,extra


#### 2. NaN 값이 있는 행에 값을 채움

In [85]:
df2 = df1.copy()

In [86]:
df2

,지역명,규모구분,연도,월,분양가격(㎡),중산층,extra
0,seoul,전체,2015,10,5841,NaN,False
1,seoul,전용면적 60㎡이하,2015,10,5652,NaN,False
2,seoul,전용면적 60㎡초과 85㎡이하,2015,10,5882,NaN,False
3,seoul,전용면적 85㎡초과 102㎡이하,2015,10,5721,NaN,False
4,seoul,전용면적 102㎡초과,2015,10,5879,NaN,False
5,인천,전체,2015,10,3163,NaN,NaN
6,인천,전용면적 60㎡이하,2015,10,3488,NaN,NaN


In [87]:
df2.fillna(value = True)

,지역명,규모구분,연도,월,분양가격(㎡),중산층,extra
0,seoul,전체,2015,10,5841,True,False
1,seoul,전용면적 60㎡이하,2015,10,5652,True,False
2,seoul,전용면적 60㎡초과 85㎡이하,2015,10,5882,True,False
3,seoul,전용면적 85㎡초과 102㎡이하,2015,10,5721,True,False
4,seoul,전용면적 102㎡초과,2015,10,5879,True,False
5,인천,전체,2015,10,3163,True,True
6,인천,전용면적 60㎡이하,2015,10,3488,True,True


In [88]:
df2

,지역명,규모구분,연도,월,분양가격(㎡),중산층,extra
0,seoul,전체,2015,10,5841,NaN,False
1,seoul,전용면적 60㎡이하,2015,10,5652,NaN,False
2,seoul,전용면적 60㎡초과 85㎡이하,2015,10,5882,NaN,False
3,seoul,전용면적 85㎡초과 102㎡이하,2015,10,5721,NaN,False
4,seoul,전용면적 102㎡초과,2015,10,5879,NaN,False
5,인천,전체,2015,10,3163,NaN,NaN
6,인천,전용면적 60㎡이하,2015,10,3488,NaN,NaN


In [ ]:
df2.fillna(value = True, inplace = True)  # 혹은 df2['extra'] = df2.fillna(value = True, inplace = True) 

In [ ]:
df2

#### 3. NaN 값이 있는 데이터를 Boolean 값으로 출력

In [89]:
pd.isna(df2)   # pd.isnull(df2)

,지역명,규모구분,연도,월,분양가격(㎡),중산층,extra
0,False,False,False,False,False,True,False
1,False,False,False,False,False,True,False
2,False,False,False,False,False,True,False
3,False,False,False,False,False,True,False
4,False,False,False,False,False,True,False
5,False,False,False,False,False,True,True
6,False,False,False,False,False,True,True


In [90]:
df2.isna()  # df2.isnull()

,지역명,규모구분,연도,월,분양가격(㎡),중산층,extra
0,False,False,False,False,False,True,False
1,False,False,False,False,False,True,False
2,False,False,False,False,False,True,False
3,False,False,False,False,False,True,False
4,False,False,False,False,False,True,False
5,False,False,False,False,False,True,True
6,False,False,False,False,False,True,True


### 연산 (Operation)

In [ ]:
df.head()

* 열(column) 기준 평균

In [ ]:
df.mean()

* 열(column) 기준 평균

In [ ]:
df.mean(axis = 1)[:5]

In [ ]:
df.head()

행을 2칸 뒤로 밀기
  -  **기본값 : axis=0 ( 행기준)**
  - 기본값 : `fill_value=None`

In [ ]:
df.shift(2)[:5]

행을 2칸 앞당기기
* 앞의 두 로우가 삭제됨

In [ ]:
df.shift(-2)[:5]

### Broadcasting을 이용한 subtract (빼기)
* `mul`, `add`, `mod`, `div`, `pow`도 지원.

In [ ]:
df1 = df[['연도', '월']]

In [ ]:
df1.head()

In [ ]:
df1.shape

In [ ]:
s = np.ones(df1.shape[0])
s.shape

In [ ]:
df1.head()

In [ ]:
s

In [ ]:
# 전체 로우에 걸쳐서 1씩 빼기연산이 됨.
df1.sub(s, axis = 0).head(n = 5)

### Apply
* np.cumsum : cumulative sum의 함수형으로 누적 합을 구하는 함수
* `apply`함수의 axis기본값은 0 즉, 로우 기준

In [ ]:
df1.apply(np.cumsum, axis = 0).head(n = 3)

In [ ]:
df1.apply(np.cumsum, axis = 0)[:3]

* 연도 column<br>
x.max() 는 최대값인 2019년도의 2019 x.min() 은 최소값인 2015년도의 2015 2019 - 2015 = 4가 출력

* 월 column<br>
x.max() 는 최대값인 12 x.min() 은 최소값인 1 12 - 1 = 11이 출력

In [ ]:
df1.head()

In [ ]:
df1.apply(lambda x: x.max() - x.min())

### 값들의 종류별 출력
    * 컬럼의 값에 대한 분포표라고 보면 됨.`R`의 `table`명령어와 동일.

In [ ]:
df1['연도'].value_counts()

In [ ]:
df1['월'].value_counts()

In [ ]:
df.규모구분.value_counts()

### 데이터 합치기 (Data Merge)

In [ ]:
df.head()

In [ ]:
left = df[['연도', '월']]
left.head()

In [ ]:
right = df['지역명']
right.head()

#### 1. Concat

In [ ]:
part1 = pd.DataFrame(np.random.randn(3, 4))
part2 = pd.DataFrame(np.random.randn(4, 4))
part3 = pd.DataFrame(np.random.randn(5, 4))

In [ ]:
part1

In [ ]:
part2

In [ ]:
part3

In [ ]:
part = [part1, part2, part3]

In [ ]:
part

In [ ]:
concat_part = pd.concat(part)

In [ ]:
concat_part.reset_index(), concat_part.shape

### 2.Join
* `Join`은 컬럼을 기주능로 합칠 경우에 `on`=`합치고자 하는 열의 이름`을 자주 사용하게 되는데 , 값이 고유하지 않다면 문제.

###### **예시1 : Key가 고유한 경우**

In [ ]:
left = pd.DataFrame({'연도':  ['2015', '2016', '2017', '2018', '2019'],
                    '월': ['1', '2', '3', '4', '5']
                    })
left

In [ ]:
right = pd.DataFrame({'이름':  ['홍길동', '김영희', '이철수', '방탄소면단', '홍준표'],
                    '월': ['1', '2', '3', '4', '5']
                    })
right

In [ ]:
left.shape, right.shape

- 매우 깔끔하게 머지가 되었다.

In [ ]:
pd.merge(left, right, on = '월')

##### 예시2 : 중복되는 키가 존재하는 경우 머지시.

In [ ]:
left = pd.DataFrame({'연도':  ['2015', '2016', '2017', '2018', '2019'],
                    '월': ['1', '1', '3', '4', '5']
                    })
left

In [ ]:
right = pd.DataFrame({'이름':  ['홍길동', '김영희', '이철수', '방탄소면단', '홍준표'],
                    '월': ['1', '1', '3', '4', '5']
                    })
right

* 행이 동일한 이름으로 2개가 생긴 것을 알수 있다.

In [ ]:
pd.merge(left, right, on = '월')

### 3.Append

In [ ]:
test = np.arange(0, 50)
test

In [ ]:
# 1d -> 2D로 변환
test = test.reshape(10, 5)
test

In [ ]:
test_1 = test[:3]

In [ ]:
test_1

In [ ]:
test_2 = test[3:7]

In [ ]:
test_2

In [ ]:
test_3 = test[7: 10]

In [ ]:
test_3

In [ ]:
df1 = pd.DataFrame(test_1)
df1

In [ ]:
df2 = pd.DataFrame(test_2)
df2

In [ ]:
df3 = pd.DataFrame(test_3)
df3

In [ ]:
df1.append(df2)

* index가 중구난방으로 되어있는데 ignore_index = <font color= red>True</font>로 해결할 수 있다.(기존 인덱스를 무시하고 새로 생성)

In [ ]:
df1.append(df2, ignore_index = True)

### 그룹화(Grouping)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.groupby(['지역명', '연도', '월'])['분양가'].agg('sum')

In [ ]:
df.info()

In [ ]:
grouped = df.groupby(['지역명', '연도', '월']).sum()
grouped

In [ ]:
grouped.head()

* 차이점을 눈으로 확인하자.
* group이 되어있는 DataFrame을 unstack의 `레벨`을 조정하여 보여줄 수 있다.위의 grouped에서 `도시` = level1, `'연도'`=level 2, '월'

In [ ]:
grouped.stack()

#### **stack & unstack을 활용한 column별 데이터 그룹핑**

In [ ]:
# level0: 열을 지역명별로 출력
grouped.unstack(0)

In [ ]:
# level1: 열을 연도별로 출력
grouped.unstack(1)

In [ ]:
# level2: 열을 월별로 출력
grouped.unstack(2)

### Pivot_table
* **pivot Table이란**
다시 말해,피벗 테이블이란 데이터 중에서 자신이 원하는 데이터만을 가지고 원하는 행과 열에 데이터를 배치하여 새로운 보고서를 만드는 기능

In [ ]:
df.head()

In [ ]:
pd.pivot_table(df, values = '분양가', index = ['연도', '지역명'], columns = '월')

In [ ]:
df.loc[(df.지역명=='강원') & (df.연도==2015)].groupby(['연도', '지역명'])['분양가'].mean()

* 월별로 보이는 가격은 분양가의 평균임을 알수있음.

### 범주형(Categoricals)

In [ ]:
df.head(10)

**분양가**에 대한 기술통계(Descritibe Statistics)를 본다면?

In [ ]:
df['분양가'].describe()

In [ ]:
df['평가'] = 0

In [ ]:
df.head(10)

가격대 별 평가

low: 25% = 2454 보다 싼 분양가

mid: 50% = 2454 ~ 3270

high: 75% = 3270 ~ 3601

very high: 75% ~ 100% = 3601 보다 비싼 분양가

### np.select를 활용하여 조건에 맞는 값을 대입하기 

In [ ]:
conditions = [
    (df['분양가'] < 2454),
    (df['분양가'] >= 2454) & (df['분양가'] < 3270),
    (df['분양가'] >= 3270) & (df['분양가'] < 3601),
    (df['분양가'] >= 3601),
    (df['분양가'] == np.nan)
]
choices = ['저렴', '보통', '비쌈', '매우 비쌈', '-']

In [ ]:
df['평가'] = np.select(conditions, choices, default = 0)

In [ ]:
df.head(20)

In [ ]:
df.groupby(by = '평가').count()

* 분양가 비싼 순으로 보기

In [ ]:
df.sort_values(by= '분양가', ascending = False)[:10]

In [ ]:
df.dtypes

In [ ]:
df['평가'] = df['평가'].astype('category')

In [ ]:
df.dtypes

In [ ]:
df.head(10)

In [ ]:
df['평가'].head()

* 범주형 값 추가 

In [ ]:
df['평가'].cat.categories=['해당없음', '개비쌈', '평균', '쫌비쌈', '쌈']

In [ ]:
df['평가'].value_counts()